In [ ]:
# upload the training_data.jsonl file to this colab file
# upload the validation_data.jsonl file to this colab file
# you should have atleast 10 lines in the each of the
# jsonl data files

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00


In [ ]:
import os
from openai import OpenAI

client = OpenAI(
  api_key=''
)


In [ ]:
import json

training_file_name = "training_data.jsonl"
validation_file_name = "validation_data.jsonl"

In [ ]:
# do not run this cell, as we are directly giving the
# jsonl data files
def prepare_data(dictionary_data, final_file_name):
    with open(final_file_name, 'w') as outfile:
        for entry in dictionary_data:
        	json.dump(entry, outfile)
        	outfile.write('\n')

prepare_data("/content/sample_data/training_data", "training_data.jsonl")
prepare_data("/content/sample_data/validation_data", "validation_data.jsonl")

In [ ]:
training_file_id = client.files.create(
  file=open("/content/sample_data/training_data.jsonl", "rb"),
  purpose="fine-tune"
)

validation_file_id = client.files.create(
  file=open("/content/sample_data/validation_data.jsonl", "rb"),
  purpose="fine-tune"
)

print(f"Training File ID: {training_file_id}")
print(f"Validation File ID: {validation_file_id}")

Training File ID: FileObject(id='file-NRe9TduqUAmuddhqwKdtVoOT', bytes=1575, created_at=1714826716, filename='training_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Validation File ID: FileObject(id='file-k1DzuO0y2HXcHtSIHWE0BKrB', bytes=1220, created_at=1714826717, filename='validation_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)


In [ ]:
response = client.fine_tuning.jobs.create(
  training_file=training_file_id.id,
  validation_file=validation_file_id.id,
  model="davinci-002",
  hyperparameters={
    "n_epochs": 15,
	"batch_size": 3,
	"learning_rate_multiplier": 0.3
  }
)
job_id = response.id
status = response.status

print(f'Fine-tunning model with jobID: {job_id}.')
print(f"Training Response: {response}")
print(f"Training Status: {status}")

Fine-tunning model with jobID: ftjob-I2dFG1UOjau0h0LVytVJKytb.
Training Response: FineTuningJob(id='ftjob-I2dFG1UOjau0h0LVytVJKytb', created_at=1714826724, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=15, batch_size=3, learning_rate_multiplier=0.3), model='davinci-002', object='fine_tuning.job', organization_id='org-xHcrqMk2YaYXl8DbAVu8rzea', result_files=[], seed=742197130, status='validating_files', trained_tokens=None, training_file='file-NRe9TduqUAmuddhqwKdtVoOT', validation_file='file-k1DzuO0y2HXcHtSIHWE0BKrB', estimated_finish=None, integrations=[], user_provided_suffix=None)
Training Status: validating_files


In [ ]:
import signal
import datetime


def signal_handler(sig, frame):
    status = client.fine_tuning.jobs.retrieve(job_id).status
    print(f"Stream interrupted. Job is still {status}.")
    return


print(f"Streaming events for the fine-tuning job: {job_id}")

signal.signal(signal.SIGINT, signal_handler)

events = client.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id)
try:
    for event in events:
        print(
            f'{datetime.datetime.fromtimestamp(event.created_at)} {event.message}'
        )
except Exception:
    print("Stream interrupted (client disconnected).")

Streaming events for the fine-tuning job: ftjob-I2dFG1UOjau0h0LVytVJKytb
2024-05-04 12:45:24 Validating training file: file-NRe9TduqUAmuddhqwKdtVoOT and validation file: file-k1DzuO0y2HXcHtSIHWE0BKrB
2024-05-04 12:45:24 Created fine-tuning job: ftjob-I2dFG1UOjau0h0LVytVJKytb


In [ ]:
import time

status = client.fine_tuning.jobs.retrieve(job_id).status
if status not in ["succeeded", "failed"]:
    print(f"Job not in terminal status: {status}. Waiting.")
    while status not in ["succeeded", "failed"]:
        time.sleep(2)
        status = client.fine_tuning.jobs.retrieve(job_id).status
        print(f"Status: {status}")
else:
    print(f"Finetune job {job_id} finished with status: {status}")
print("Checking other finetune jobs in the subscription.")
result = client.fine_tuning.jobs.list()
print(f"Found {len(result.data)} finetune jobs.")

Job not in terminal status: validating_files. Waiting.
Status: validating_files
Status: validating_files
Status: queued
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
